In [1]:
import numpy as np
import pandas as pd
import spacy

In [2]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()


In [3]:
sent_df=pd.read_csv('../Downloads/archive (6)/train.csv',encoding='unicode_escape',usecols=['text','sentiment'])

In [4]:
from nltk.tokenize import word_tokenize
import re


In [5]:
sent_df.dropna(inplace=True)

In [6]:
sent_df

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative
...,...,...
27476,wish we could come see u on Denver husband l...,negative
27477,I`ve wondered about rake to. The client has ...,negative
27478,Yay good for both of you. Enjoy the break - y...,positive
27479,But it was worth it ****.,positive


In [7]:
sent_df['sentiment']=le.fit_transform(sent_df['sentiment'])

In [8]:
sent_df

,text,sentiment
0,"I`d have responded, if I were going",1
1,Sooo SAD I will miss you here in San Diego!!!,0
2,my boss is bullying me...,0
3,what interview! leave me alone,0
4,"Sons of ****, why couldn`t they put them on t...",0
...,...,...
27476,wish we could come see u on Denver husband l...,0
27477,I`ve wondered about rake to. The client has ...,0
27478,Yay good for both of you. Enjoy the break - y...,2
27479,But it was worth it ****.,2


In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(sent_df['text'],sent_df['sentiment'],test_size=0.2)

In [10]:
import string
exclude=string.punctuation
def punc_rem(text):
    return text.translate(str.maketrans('','',exclude)).strip()

In [11]:
X_train=X_train.apply(punc_rem)
X_test=X_test.apply(punc_rem)

In [12]:
X_train=X_train.str.lower()
X_test=X_test.str.lower()

In [13]:
import re
exc=re.compile(r'\d+')
def number_adj(txt):
    return re.sub(exc,'',txt)

In [14]:
X_train=X_train.apply(number_adj)
X_test=X_test.apply(number_adj)

In [15]:
from textblob import TextBlob


In [22]:
def word_correction(text):
    txt=TextBlob(text).correct()
    return txt.string

In [23]:
word_correction(X_train.iloc[16627])

'we travelled the world  i grew up on a diet of everything you can think of still my geese grand made her own past'

In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [3]:
analyzer = SentimentIntensityAnalyzer()

# Function to apply VADER sentiment analysis
def analyze_sentiment_vader(text):
    return analyzer.polarity_scores(text)['compound']

0.4215